# Fine-Tuning 

Referred to code from [Link](https://stabilityai.notion.site/Stable-Diffusion-3-Medium-Fine-tuning-Tutorial-17f90df74bce4c62a295849f0dc8fb7e).

## Setup

In [ ]:
# Check the GPU
!nvidia-smi

In [ ]:
# Install dependencies.
!pip install bitsandbytes transformers accelerate peft -q

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git -q

Download diffusers SDXL DreamBooth training script.

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

## Dataset

Here ``face_dataset.zip`` contains the images (in png or jpg format) and captions (in a txt file).

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols, resize=256):

    if resize is not None:
        imgs = [img.resize((resize, resize)) for img in imgs]
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

In [ ]:
!unzip face_dataset.zip

In [ ]:
!mv face_dataset bajirao_face_dataset

In [ ]:
import glob

# change path to display images from your local dir
img_paths = "./bajirao_face_dataset/*.jpg"
imgs = [Image.open(path) for path in glob.glob(img_paths)]

num_imgs_to_preview = 5
image_grid(imgs[:num_imgs_to_preview], 1, num_imgs_to_preview)

In [ ]:
import glob
from PIL import Image

# create a list of (Pil.Image, path) pairs
local_dir = "./bajirao_face_dataset/"
imgs_and_paths = [(path,Image.open(path)) for path in glob.glob(f"{local_dir}*.jpg")]

In [ ]:
import json

with open(f'{local_dir}metadata.jsonl', 'w') as outfile:
  for img in imgs_and_paths:
      print(img)
      with open(img[0].split(".")[0]+".txt",'rb') as f:
        caption = str(f.read())
      entry = {"file_name":img[0].split("/")[-1], "prompt": caption}
      json.dump(entry, outfile)
      outfile.write('\n')

## Prep for training

Initialize `accelerate`:

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!accelerate config default

### Log into your Hugging Face account
Pass [your **write** access token](https://huggingface.co/settings/tokens) so that we can push the trained checkpoints to the Hugging Face Hub:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Train

#### Set Hyperparameters
To ensure we can DreamBooth with LoRA on a heavy pipeline like Stable Diffusion XL, we're using:

* Gradient checkpointing (`--gradient_accumulation_steps`)
* 8-bit Adam (`--use_8bit_adam`)
* Mixed-precision training (`--mixed-precision="fp16"`)

### Launch training

To allow for custom captions we need to install the `datasets` library, you can skip that if you want to train solely
 with `--instance_prompt`.
In that case, specify `--instance_data_dir` instead of `--dataset_name`

In [ ]:
!pip install datasets -q

 - Use `--output_dir` to specify your LoRA model repository name!
 - Use `--caption_column` to specify name of the cpation column in your dataset. In this example we used "prompt" to
 save our captions in the
 metadata file, change this according to your needs.

In [ ]:
# prompt: write script to remove all the .txt files in a folder (txt file consists of a single line captions)

import os

def remove_txt_files(folder_path):
  """Removes all .txt files from the specified folder.

  Args:
    folder_path: The path to the folder containing the .txt files.
  """
  for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
      file_path = os.path.join(folder_path, filename)
      try:
        if os.path.isfile(file_path):
          os.remove(file_path)
          print(f"Removed: {file_path}")
      except Exception as e:
        print(f"Error removing {file_path}: {e}")

# Example usage (replace with your actual folder path):
folder_to_clean = "bajirao_face_dataset"
remove_txt_files(folder_to_clean)


Training for a maximum of ``1500`` steps.

In [ ]:
#!/usr/bin/env bash
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --output_dir="bajirao_LORA" \
  --instance_prompt="a photo of sks Bajirao" \
  --caption_column="prompt"\
  --mixed_precision="fp16" \
  --dataset_name="bajirao_face_dataset" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=3 \
  --gradient_checkpointing \
  --learning_rate=1e-4 \
  --snr_gamma=5.0 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --max_train_steps=1500 \
  --checkpointing_steps=717 \
  --seed="0"

In [ ]:
!zip -r bajirao_LORA.zip bajirao_LORA

*Note:* This zip file consists of the fine-tuned weights.